In [1]:
import random
import h5py
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
import time
import tensorflow as tf
from sklearn.metrics import roc_curve, roc_auc_score
from tensorflow.keras.utils import to_categorical
import tensorflow_addons as tfa
import os
import time
import gc
import copy



In [2]:
electron_path = '/kaggle/input/electron-photon-classifier/electron/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'
photon_path = '/kaggle/input/electron-photon-classifier/photon/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'

electron = h5py.File(electron_path, "r")
print([x for x in electron])
print(electron['X'],"\n",electron['y'])
print(electron['X'][0].shape)
print((electron['y'][:]))
photon = h5py.File(photon_path, "r")
print([x for x in photon])
print(photon['X'],"\n",photon['y'])
print(photon['X'][0].shape)
print((photon['y'][:]))

['X', 'y']
<HDF5 dataset "X": shape (249000, 32, 32, 2), type "<f4"> 
 <HDF5 dataset "y": shape (249000,), type "<f4">
(32, 32, 2)
[1. 1. 1. ... 1. 1. 1.]
['X', 'y']
<HDF5 dataset "X": shape (249000, 32, 32, 2), type "<f4"> 
 <HDF5 dataset "y": shape (249000,), type "<f4">
(32, 32, 2)
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
def plot_pics(num,cmapclr,cmapclr_):
    el1 = electron['X'][num][:,:,0]
    el2 = electron['X'][num][:,:,1]

    ph1 = photon['X'][num][:,:,0]
    ph2 = photon['X'][num][:,:,1]

    plt.figure(figsize = (16,16))
    
    plt.subplot(1,4,1)
    plt.title('Hit energy e-')
    plt.imshow(el1, cmap=cmapclr, alpha=1)
    plt.subplot(1,4,2)
    plt.title("Time e-")
    plt.imshow(el2, cmap = cmapclr_, alpha=1)

    plt.subplot(1,4,3)
    plt.title('Hit energy ph')
    plt.imshow(ph1, cmap = cmapclr, alpha=1)
    plt.subplot(1,4,4)
    plt.title("Time ph")
    plt.imshow(ph2, cmap = cmapclr_, alpha=1)

xy = ['Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'winter', 'winter_r']

print(len(xy))


In [4]:
def plotV2(arr,x,cmapclr,cmapclr_):
    image_ch1 = np.array((arr)[x])[:,:,0]
    image_ch2 = np.array((arr)[x])[:,:,1]
    plt.figure(figsize = (16,16))
    plt.subplot(1,2,1)
    plt.title('Hit energy e-')
    plt.imshow(image_ch1, cmap=cmapclr, alpha=1)
    plt.subplot(1,2,2)
    plt.title("Time e-")
    plt.imshow(image_ch2, cmap = cmapclr_, alpha=1)
# plotV2(trainnn,12,'gray_r','gray_r')
# plotV2(train,12,'gray_r','gray_r')


In [ ]:
def shuffle_and_distributeLabelsV2():
    global train
    global train_ultimate
    global train_label
    np.random.shuffle(train_ultimate)
    train = [x[0] for x in train_ultimate]
    train_label = [x[1] for x in train_ultimate]

In [13]:
def normalise_bw_01(arr):
    arr = np.array(arr)
    arr = arr.reshape(-1)
    print(f"Before min:{np.min(arr)} max:{np.max(arr)}")
    global_min = np.min(arr)
    arr+= -global_min
    global_max = np.max(arr)
    arr/= global_max
    arr = arr.reshape(498000, 32, 32, 2)
    print(f"After min:{np.min(arr)} max:{np.max(arr)}")
    return arr

In [28]:
def plot_Model_loss():
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.show()

In [29]:
def plot_Model_accuracy():
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

In [ ]:
def plot_Model_RocAucV2():
    predicted_labels = model.predict(test_data)

    fpr, tpr, thresholds = roc_curve(test_labels, predicted_labels)
    roc_auc = roc_auc_score(test_labels, predicted_labels)

    # Plot the ROC curve for the positive class
    plt.figure(figsize=(8,6))
    plt.plot(fpr, tpr, color='green', lw=1,label='ROC curve (AUC = {0:0.2f})'''.format(roc_auc))

    # Plot the diagonal line for reference
    plt.plot([0, 1], [0, 1], 'k--', lw=1)

    # Set the limits and labels of the plot
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC AUC Curve for Binary Classification')
    plt.legend(loc="lower right")
    plt.show()


In [30]:
def plot_Model_lossAcc(his):
    plt.figure(figsize = (16,8))

    plt.subplot(2,2,1)
    plt.plot(his.history['loss'])
    plt.plot(his.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.show()
    
    plt.figure(figsize = (16,8))

    plt.subplot(2,2,2)
    plt.plot(his.history['accuracy'])
    plt.plot(his.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()
    


In [32]:
def plot_Model_RocAuc():
    predicted_labels = model.predict(test_data)

    y_test_binary = to_categorical(test_labels)
    y_pred_binary = predicted_labels

    # Calculate the ROC curves and their AUC scores for each class
    fpr = {}
    tpr = {}
    roc_auc = {}
    n_classes = y_test_binary.shape[1]
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred_binary[:, i])
        roc_auc[i] = roc_auc_score(y_test_binary[:, i], y_pred_binary[:, i])

    # Compute micro-average ROC curve and ROC AUC score
    fpr_micro, tpr_micro, _ = roc_curve(y_test_binary.ravel(), y_pred_binary.ravel())
    roc_auc_micro = roc_auc_score(y_test_binary, y_pred_binary)

    # Plot the ROC curves for each class and the micro-average ROC curve
    plt.figure(figsize=(8,6))
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=1,label='ROC curve of class {0} (AUC = {1:0.2f})'.format(i, roc_auc[i]))
    # plt.plot(fpr_micro, tpr_micro, color='deeppink', lw=1,
    #          label='Micro-average ROC curve (AUC = {0:0.2f})'
    #          ''.format(roc_auc_micro))

    # Plot the diagonal line for reference
    plt.plot([0, 1], [0, 1], 'k--', lw=1)

    # Set the limits and labels of the plot
    # plt.xlim([-0.05, 1.0])
    # plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC AUC Curve for Multiclass Classification')
    plt.legend(loc="lower right")
    plt.show()

In [5]:
gc.collect()

46

In [6]:
electron_arre = copy.deepcopy(electron['X'][:])
photon_arre = copy.deepcopy(photon['X'][:])
both = []
both.extend(electron_arre)
both.extend(photon_arre)

In [7]:
electron_arrere = copy.deepcopy(electron['y'][:])
photon_arrere = copy.deepcopy(photon['y'][:])
both_re = []
both_re.extend(electron_arrere)
both_re.extend(photon_arrere)

In [ ]:
del electron_arre,photon_arre, photon_arrere, electron_arrere
gc.collect()

In [8]:
both = np.array(both)
both_re = np.array(both_re)
print(both.shape,both_re.shape)

(498000, 32, 32, 2) (498000,)


In [9]:
gc.collect()

92

In [10]:
train_ultimate = [x for x in zip(both,both_re)]
train = []
train_label = []
electron.close()
photon.close()

In [11]:
del both,both_re
gc.collect()

0

In [14]:
shuffle_and_distributeLabelsV2()
train = normalise_bw_01(train)

Before min:-2.512557029724121 max:2.2779698371887207
After min:0.0 max:1.0


In [15]:
# print(both.shape,both_re.shape,np.array(train_ultimate,dtype = object).shape,len(train_ultimate),len(train),train_label[:10])

In [16]:
del train_ultimate
gc.collect()

115

In [20]:
# trainV2 = copy.deepcopy(train)
# trainV2 = np.array(trainV2[:])
# convert_bw_01(trainV2)


In [21]:
sumnum = 200000
sumnum2 = 200000
train2 = copy.deepcopy(train[:int(sumnum+sumnum2)])
train_label2 = copy.deepcopy(train_label[:int(sumnum+sumnum2)])

In [22]:

train = train2[:sumnum]
train_label = train_label2[:sumnum]

In [23]:
del train,train_label
gc.collect()

23

In [24]:
train = train2[sumnum2:]
train_label = train_label2[sumnum2:]

In [25]:
val_size = 0.2
test_size = 0.1

val_data_size = int(len(train[:sumnum])*val_size)
test_data_size = int(len(train[:sumnum])*test_size)
train_data_size = len(train[:sumnum])-val_data_size-test_data_size

train_data = train[:train_data_size]
train_labels = train_label[:train_data_size]

val_data = train[train_data_size:train_data_size+test_data_size]
val_labels = train_label[train_data_size:train_data_size+test_data_size]

test_data = train[train_data_size+test_data_size:]
test_labels = train_label[train_data_size+test_data_size:]

In [26]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)

val_data = np.array(val_data)
val_labels = np.array(val_labels)

test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [27]:
print(len(val_data),len(val_labels))
print(val_data_size,test_data_size,train_data_size,len(train))
print()
# print(train_label[train_data_size:test_data_size])

20000 20000
40000 20000 140000 200000



In [33]:
# metrics=[tf.keras.metrics.TruePositives(name= 'tp'),tf.keras.metrics.FalsePositives(name='fp'),tf.keras.metrics.TrueNegatives(name='tn'),tf.keras.metrics.FalseNegatives(name='fn'),tf.keras.metrics.BinaryAccuracy(name='accuracy'),tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall'),tf.keras.metrics.AUC(name='auc')]
# print(metrics)

[<keras.metrics.metrics.TruePositives object at 0x7f109d3278d0>, <keras.metrics.metrics.FalsePositives object at 0x7f109d327e10>, <keras.metrics.metrics.TrueNegatives object at 0x7f10a349e8d0>, <keras.metrics.metrics.FalseNegatives object at 0x7f10a298d050>, <keras.metrics.metrics.BinaryAccuracy object at 0x7f10a298d790>, <keras.metrics.metrics.Precision object at 0x7f109e4e0190>, <keras.metrics.metrics.Recall object at 0x7f109e4e07d0>, <keras.metrics.metrics.AUC object at 0x7f10a0258050>]


In [34]:
# del model
# del history
gc.collect()

23

In [35]:
# initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.,seed = 121)
# layer = tf.keras.layers.Dense(3, kernel_initializer=initializer)
# ,kernel_regularizer=tf.keras.regularizers.l1(0.01))
# ,kernel_regularizer=tf.keras.regularizers.l2(0.01))
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3,3),strides = (1,1), padding = 'valid', activation = 'relu', input_shape = (32,32,2)))
model.add(tf.keras.layers.Conv2D(12,(3,3),strides = (1,1), padding = 'valid', activation = 'relu'))
model.add(tf.keras.layers.GroupNormalization(groups=12))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides = None, padding = 'valid'))
# model.add(tf.keras.layers.Conv2D(64,(3,3), strides = (1,1), padding = 'valid', activation = 'relu'))
model.add(tf.keras.layers.Conv2D(8,(3,3),strides = (1,1), padding = 'valid', activation = 'relu'))
model.add(tf.keras.layers.GroupNormalization(groups=8))

model.add(tf.keras.layers.Conv2D(8, (3,3),strides = (1,1), padding = 'valid', activation = 'relu'))
model.add(tf.keras.layers.GroupNormalization(groups=8))
# model.add(tf.keras.layers.Conv2D(8, (3,3),strides = (1,1), padding = 'valid', activation = 'relu'))
# model.add(tf.keras.layers.Conv2D(8, (3,3),strides = (1,1), padding = 'valid', activation = 'relu'))
# model.add(tf.keras.layers.GroupNormalization(groups=8))
model.add(tf.keras.layers.Conv2D(6, (3,3),strides = (1,1), padding = 'valid', activation = 'relu'))
model.add(tf.keras.layers.GroupNormalization(groups=6))

model.add(tf.keras.layers.Conv2D(6, (3,3),strides = (1,1), padding = 'valid', activation = 'relu'))
model.add(tf.keras.layers.GroupNormalization(groups=6))

model.add(tf.keras.layers.Conv2D(6, (3,3),strides = (1,1), padding = 'valid', activation = 'relu'))
model.add(tf.keras.layers.GroupNormalization(groups=6))

# model.add(tf.keras.layers.Conv2D(64, (3,3),strides = (1,1), padding = 'valid', activation = 'relu'))
# model.add(tf.keras.layers.Conv2D(64, (3,3),strides = (1,1), padding = 'valid', activation = 'relu'))

# model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides = None, padding = 'valid'))
# model.add(tf.keras.layers.Conv2D(8,(3,3), strides = (1,1), padding = 'valid', activation = 'relu'))
# model.add(tf.keras.layers.Conv2D(4,(3,3), strides = (1,1), padding = 'valid', activation = 'relu'))


model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Normalization())
# model.add(tf.keras.layers.Dropout(0.3))
# model.add(tf.keras.layers.Dense(1024,activation = 'relu'))
# model.add(tf.keras.layers.Dense(512,activation = 'relu'))

# model.add(tf.keras.layers.LayerNormalization())
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64,activation = 'relu'))
model.add(tf.keras.layers.Normalization())

model.add(tf.keras.layers.Dense(128,activation = 'relu'))
model.add(tf.keras.layers.Normalization())

# model.add(tf.keras.layers.Dense(128,activation = 'relu'))

model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))


### sparse_categorical_crossentropy, binary_crossentropy

In [36]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.build(input_shape = (32,32,2))

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 16)        304       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 12)        1740      
                                                                 
 group_normalization (GroupN  (None, 28, 28, 12)       24        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 12)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 8)         872       
                                                                 
 group_normalization_1 (Grou  (None, 12, 12, 8)        1

In [38]:
epochs = 50
checkpoint_filepath = '/kaggle/working/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_accuracy')
EarlyStop = tf.keras.callbacks.EarlyStopping(monitor='EarlyStopping', patience=5)
def scheduler(epoch, lr):
    flr = 1e-3
    if epoch < epochs*0.1 :
        return flr
#     elif epoch > epochs*0.1 and epoch < epochs*0.25:
#         lr *= tf.math.exp(-0.11)
#         return lr
    else:
        lr *= tf.math.exp(-0.008)
#         return lr * tf.math.exp(-0.004)
        return flr
learning_rate_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
history = model.fit(train_data,train_labels,epochs=epochs,batch_size = 500,callbacks=[model_checkpoint_callback,EarlyStop,learning_rate_callback],validation_data=(val_data,val_labels),shuffle=True,initial_epoch = 0)

Epoch 1/50


In [ ]:
# plot_Model_loss()
# plot_Model_accuracy()
plot_Model_lossAcc(history)
plot_Model_RocAucV2()

In [ ]:
# model.save('/kaggle/working/model777',include_optimizer=True)

In [ ]:
# import shutil
# shutil.make_archive('recentbest777', 'zip', '/kaggle/working/model5')

In [ ]:
!ls /kaggle/working/

In [ ]:
# def schedulerV2(epoch, lr):
#     flr = lr = 9.802e-4
#     if epoch < 10 :
#         return lr
# #     elif epoch > 10 and epoch <25:
# #         return flr * tf.math.exp(0.01)
#     else:
#         return lr * tf.math.exp(-0.01)
# learning_rate_callbackV2 = tf.keras.callbacks.LearningRateScheduler(schedulerV2)

In [ ]:
# history2 = model.fit(train_data,train_labels,epochs=100,batch_size = 500,callbacks=[model_checkpoint_callback,learning_rate_callbackV2],validation_data=(val_data,val_labels),shuffle=True)

In [ ]:

# plot_Model_lossAcc(history2)
# plot_Model_RocAucV2()